In [2]:
#  functions
import pypsa, pandas as pd, numpy as np
print("PyPSA:", pypsa.__version__)
def summarize(n):
    tables = ["buses","loads","generators","lines","links","transformers","storage_units"]
    counts = {t: getattr(n, t).shape[0] for t in tables}
    print(f"Network: {getattr(n, 'name', 'unknown')}")
    print("Snapshots:", len(getattr(n, "snapshots", [])))
    for t,c in counts.items():
        print(f"{t:14s}: {c}")
    if hasattr(n, "carriers") and not n.carriers.empty:
        print("\nCarriers defined:", ", ".join(n.carriers.index))
    else:
        print("\nCarriers defined: none")

PyPSA: 0.35.2


In [11]:
n = pypsa.Network()
n.name = "toy-24h"
# time snapshots, add number of shots, define yearly scale
hours = pd.date_range("2025-01-01", periods=24, freq="h")
n.set_snapshots(hours)
H=len(n.snapshots)
w = 8760/H

# define carriers
n.add("Carrier","electricity")
n.add("Carrier","wind")
n.add("Carrier","gas")
n.add("Carrier","battery")  
n.add("Carrier", "coal")
n.add("Carrier", "hydro")
# add bus
n.add("Bus","bus0", carrier="electricity")

# add load, match snapshots, assume sin pattern, vary 5 MW to 15 MW throughout day
hangles = np.linspace(0, 2*np.pi, H, endpoint=False)
p = 10 + 5*np.sin(hangles)
n.add("Load","demand", bus="bus0",
      p_set=p)

# add wind generator, small random marginal cost, random loads between 0 and 1, scale CAPEX to daily
rng = np.random.default_rng(42)
n.add("Generator","wind", bus="bus0", carrier="wind",
      p_max_pu=rng.random(H),
      p_nom_extendable=True,
      marginal_cost=2,
      capital_cost=900)

# add gas generator, built for backup, high marginal cost
n.add("Generator","ocgt", bus="bus0", carrier="gas",
      p_nom_extendable=True,
      marginal_cost=70,
      capital_cost=500)

#  add battery storage, high efficiency
n.add("StorageUnit","battery", bus="bus0",
      p_nom_extendable=True, max_hours=2,
      efficiency_store=0.95, efficiency_dispatch=0.95,
      carrier="battery",
      capital_cost=120)
#add coal power plant (high capital cost, lower marginal than gas, stable load, include variation)
n.add(
    "Generator", "coal_pp",
    bus="bus0", carrier="coal",
    p_nom_extendable=True,
    capital_cost=650,
    marginal_cost=45.0,
    p_min_pu=0.50,
    ramp_limit_up=0.10,
    ramp_limit_down=0.10
)
# add hydro reservoir, high capital cost, no fuel cost
n.add("StorageUnit", "hydro_dam", bus="bus0", carrier="hydro",
      p_nom_extendable=True,
      max_hours=90,     #90 hour reservoir
      efficiency_store=1.0,
      efficiency_dispatch=0.90, #loss during conversion
      standing_loss=0.001,         # evaporation loss
      capital_cost=1000,
      marginal_cost=0.0)

#model inflow of dam
inflow = pd.Series(5.0 + 3.0*np.sin(hangles),
                   index=n.snapshots)

# assign the inflow to the dam
n.storage_units_t.inflow.loc[:, "hydro_dam"] = inflow
# optimize free variables
n.optimize(solver_name="highs")

#print energy balance checker
balance = (n.generators_t.p.sum(axis=1)
           + n.storage_units_t.p_dispatch.sum(axis=1)
           - n.storage_units_t.p_store.sum(axis=1)
           - n.loads_t.p.sum(axis=1)).round(6)
print("\nPower balance residual (should be 0 each hour):")
print(balance.head(9))

# first 9 hours for all storage units
print("\nStorage discharge p_dispatch (MW), first 9h:")
print(n.storage_units_t.p_dispatch.head(9))
print("\nStorage charge p_store (MW), first 9h:")
print(n.storage_units_t.p_store.head(9))
print("\nStorage state of charge (MWh), first 9h:")
print(n.storage_units_t.state_of_charge.head(9))

#print size and dispatch
print("Optimal capacities (MW):\n", n.generators.p_nom_opt)
print("\nDispatch (first 9 hours, MW):\n", n.generators_t.p.head(9))

#print hydro inflow
if "hydro_dam" in n.storage_units.index:
    print("\n[hydro_dam] inflow (MW), first 9h:")
    print(n.storage_units_t.inflow["hydro_dam"].head(9))


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.09s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 245 primals, 555 duals
Objective: 1.73e+04
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Generator-ext-p-ramp_limit_up, Generator-ext-p-ramp_limit_down, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-myou24os has 555 rows; 245 cols; 1197 nonzeros
Coefficient ranges:
  Matrix [6e-02, 9e+01]
  Cost   [2e+00, 1e+03]
  Bound  [2e+00, 8e+00]
  RHS    [2e+00, 2e+01]
Presolving model
358 rows, 245 cols, 1000 nonzeros  0s
Dependent equations search running on 71 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
357 rows, 244 cols, 998 nonzeros  0s
Presolve : Reductions: rows 357(-198); columns 244(-1); elements 998(-199)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0800000000e+06 Ph1: 72(75976); Du: 24(1080) 0s
        220     1.7297279097e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-myou24os
Model status        : Optimal
Simplex   iterations: 220
Obj